## AI Inference from a video file into a bunch of images
This notebook is an example how to use DeGirum PySDK to do AI inference of a video stream from a video file. The annotated frames are saved into new directory with `_annotated` suffix.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you just need to uncomment **one** of the lines in the code below.

You also need to specify your cloud API access token, cloud zoo URLs, and AI server hostname in [env.ini](env.ini) file, located in the same directory as this notebook.


#### Specify video file name here

In [ ]:
input_filename = "./images/Traffic.mp4" # video file to process

#### Specify where do you want to run your inferences

In [ ]:
import degirum as dg, mytools

cloud_token = mytools.get_token() # get cloud API access token from env.ini file
cloud_zoo_url = mytools.get_cloud_zoo_url() # get cloud zoo URL from env.ini file

#
# Please UNCOMMENT only ONE of the following lines to specify where to run AI inference
#

# 1. Inference on the DeGirum Cloud Platform
zoo = dg.connect(dg.CLOUD, cloud_zoo_url, cloud_token)

# 2. Inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN
# zoo = dg.connect(mytools.get_ai_server_hostname(), cloud_zoo_url, cloud_token)

# 3. Inference on DeGirum ORCA accelerator installed on your computer
# zoo = dg.connect(dg.LOCAL, cloud_zoo_url, cloud_token)

#### The rest of the cells below should run without any modifications

In [ ]:
import cv2
from pathlib import Path

In [ ]:
# load object detection model
# (change model name to "...n2x_cpu_1" to run it on CPU)
model = zoo.load_model("mobilenet_v2_ssd_coco--300x300_quant_n2x_orca_1")

# set model parameters
model.image_backend = 'opencv' # select OpenCV backend: needed to have overlay image in OpenCV format
model.input_numpy_colorspace = 'BGR'
model.overlay_show_probabilities = True # show probabilities on overlay image
model.overlay_line_width = 1

In [ ]:
orig_path = Path(input_filename)
frame_path = orig_path.parent / (orig_path.name + "_original") # construct original images directory path

# decode video stream into a set of jpeg images
nframes = mytools.video2jpegs(input_filename, frame_path)
print(f"{nframes} images are saved into '{frame_path}'")

In [ ]:
ann_path = orig_path.parent / (orig_path.name + "_annotated") # construct annotated images directory path

if not ann_path.exists(): # create directory for annotated images
    ann_path.mkdir()
    
# AI prediction loop
fi = 0 # frame index
print("Processing frames:")
progress = mytools.Progress(nframes)
for res in model.predict_dir(str(frame_path)):
    fname = str(ann_path / f"frame_{fi:05d}.jpg")
    cv2.imwrite(fname, res.image_overlay)
    fi += 1
    progress.step()
print(f"{nframes} annotated images are saved into '{ann_path}'")